In [80]:
submission.subscribe.value_counts()

0    7150
1     350
Name: subscribe, dtype: int64

In [79]:
submission = submission.rename(columns={0:'subscribe'})
submission.head()

,id,subscribe
subscribe,22501,0
1,22502,0
2,22503,0
3,22504,0
4,22505,0


In [77]:
submission.drop('subscribe',axis=1,inplace=True)
submission = submission.rename({0:'subscribe'})

In [71]:
#预测
submission = submission.join(pd.DataFrame(RF.predict(test)))
submission.head()

ValueError: columns overlap but no suffix specified: Index([0], dtype='object')

In [72]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         7500 non-null   int64 
 1   subscribe  7500 non-null   object
 2   0          7500 non-null   int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 146.6+ KB


In [57]:
# 哑变量转换 train
test_job = pd.get_dummies(test['job'], prefix='job', drop_first=True)
test_mar = pd.get_dummies(test['marital'], prefix='mar', drop_first=True)
test_edu = pd.get_dummies(test['education'], prefix='edu', drop_first=True)
test_h = pd.get_dummies(test['housing'], prefix='h', drop_first=True)
test_loan = pd.get_dummies(test['loan'], prefix='loan', drop_first=True)
test_def = pd.get_dummies(test['default'], prefix='def', drop_first=True)
test_pou = pd.get_dummies(test['poutcome'], prefix='pou', drop_first=True)
test = test.join([test_job,test_mar,test_edu,test_h,test_loan,
                  test_def,test_pou])
test.drop(['job','marital','education','housing','loan','default','poutcome'],inplace=True,axis=1)
test.head(2)

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed,...,edu_6,edu_7,h_1,h_2,loan_1,loan_2,def_1,def_2,pou_1,pou_2
0,35,3295,1,476,0,1.4,95.37,-33.04,3.63,5204.54,...,0,0,0,1,0,1,0,0,0,1
1,26,2872,1,166,2,-1.8,91.75,-44.42,3.16,4924.78,...,0,0,0,1,0,0,0,0,0,1


In [56]:
# 数据编码
from sklearn.preprocessing import LabelEncoder
for i in test.columns:
    if test[i].dtype == 'object':
        test[i] = LabelEncoder().fit_transform(test[i])
test.head()

,age,job,marital,education,default,housing,loan,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed
0,35,9,2,5,0,2,2,3295,1,476,0,2,1.4,95.37,-33.04,3.63,5204.54
1,26,0,2,3,0,2,0,2872,1,166,2,2,-1.8,91.75,-44.42,3.16,4924.78
2,44,1,1,1,0,0,0,268,3,968,0,1,1.1,89.67,-36.90,5.04,4947.02
3,36,1,1,2,0,2,0,30,1,432,5,2,-0.1,89.87,-41.66,3.27,5203.33
4,41,1,1,0,0,2,0,1670,1,944,3,2,1.1,97.64,-36.32,3.95,4992.02


In [55]:
# 测试集处理
test = test.drop(['id','contact','month','day_of_week',],axis=1)

In [49]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
LR3 = LogisticRegression()
# 如果没有指定机器学习器，则
bag = BaggingClassifier(base_estimator=LR3,n_estimators=50)
bag.fit(Xtrain,Ytrain)   # 计算
print(classification_report(Ytest, bag.predict(Xtest)))

BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=50)

              precision    recall  f1-score   support

           0       0.87      0.99      0.93      4887
           1       0.51      0.06      0.11       738

    accuracy                           0.87      5625
   macro avg       0.69      0.53      0.52      5625
weighted avg       0.83      0.87      0.82      5625



In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
#先实例一个机器学习器
LR2 = LogisticRegression()
# 如果没有指定机器学习器，则默认为决策树（最大深度为1）
ada= AdaBoostClassifier(base_estimator= LR2, n_estimators= 20)  # 实例化
ada.fit(Xtrain,Ytrain)   # 计算
print(classification_report(Ytest, ada.predict(Xtest)))

AdaBoostClassifier(base_estimator=LogisticRegression(), n_estimators=20)

              precision    recall  f1-score   support

           0       0.87      0.99      0.93      4887
           1       0.49      0.05      0.09       738

    accuracy                           0.87      5625
   macro avg       0.68      0.52      0.51      5625
weighted avg       0.82      0.87      0.82      5625



In [41]:
# 导入 决策树
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(Xtrain,Ytrain)
DT.score(Xtest,Ytest)
print(classification_report(Ytest, DT.predict(Xtest)))

DecisionTreeClassifier()

0.8364444444444444

              precision    recall  f1-score   support

           0       0.91      0.90      0.91      4887
           1       0.39      0.42      0.40       738

    accuracy                           0.84      5625
   macro avg       0.65      0.66      0.65      5625
weighted avg       0.84      0.84      0.84      5625



In [39]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(Xtrain,Ytrain)
knn.score(Xtest,Ytest)
print(classification_report(Ytest, knn.predict(Xtest)))

KNeighborsClassifier(n_neighbors=3)

AttributeError: 'NoneType' object has no attribute 'split'

In [34]:
from sklearn.ensemble import RandomForestClassifier
# 先用默认参数，后续可用交叉验证，bootstrap:是否随机采样
RF = RandomForestClassifier(bootstrap=True,oob_score=True) #袋外样本必需设置True，可以验证
RF.fit(Xtrain,Ytrain)
RF.score(Xtest,Ytest)
print(classification_report(Ytest, RF.predict(Xtest)))

RandomForestClassifier(oob_score=True)

0.8773333333333333

              precision    recall  f1-score   support

           0       0.89      0.98      0.93      4887
           1       0.59      0.21      0.31       738

    accuracy                           0.88      5625
   macro avg       0.74      0.59      0.62      5625
weighted avg       0.85      0.88      0.85      5625



In [33]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier()
xgb.fit(Xtrain,Ytrain)
#.这个没有score
# 模型评估
from sklearn.metrics import classification_report
print(classification_report(Ytest, xgb.predict(Xtest)))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

              precision    recall  f1-score   support

           0       0.91      0.96      0.93      4887
           1       0.54      0.34      0.42       738

    accuracy                           0.88      5625
   macro avg       0.72      0.65      0.67      5625
weighted avg       0.86      0.88      0.86      5625



In [32]:
# 建立模型
from lightgbm import LGBMClassifier
LGBM = LGBMClassifier()
LGBM.fit(Xtrain,Ytrain)
# 模型评估
from sklearn.metrics import classification_report
print(classification_report(Ytest, LGBM.predict(Xtest)))

LGBMClassifier()

              precision    recall  f1-score   support

           0       0.91      0.96      0.93      4887
           1       0.58      0.34      0.43       738

    accuracy                           0.88      5625
   macro avg       0.74      0.65      0.68      5625
weighted avg       0.86      0.88      0.87      5625



In [31]:
# 哑变量转换 test
dummy_job_t = pd.get_dummies(Xtest['job'], prefix='job', drop_first=True)
dummy_mar_t = pd.get_dummies(Xtest['marital'], prefix='mar', drop_first=True)
dummy_edu_t = pd.get_dummies(Xtest['education'], prefix='edu', drop_first=True)
dummy_h_t = pd.get_dummies(Xtest['housing'], prefix='h', drop_first=True)
dummy_loan_t = pd.get_dummies(Xtest['loan'], prefix='loan', drop_first=True)
dummy_def_t = pd.get_dummies(Xtest['default'], prefix='def', drop_first=True)
dummy_pou_t = pd.get_dummies(Xtest['poutcome'], prefix='pou', drop_first=True)
Xtest = Xtest.join([dummy_job_t,dummy_mar_t,dummy_edu_t,dummy_h_t,dummy_loan_t,
                      dummy_def_t,dummy_pou_t])
Xtest.drop(['job','marital','education','housing','loan','default','poutcome'],inplace=True,axis=1)
Xtest.head()

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed,...,edu_6,edu_7,h_1,h_2,loan_1,loan_2,def_1,def_2,pou_1,pou_2
10110,46,164,3,995,0,1.4,96.07,-42.27,4.74,5170.59,...,1,0,0,0,0,1,0,0,1,0
7865,38,258,1,959,0,1.4,97.67,-42.25,4.89,5013.74,...,0,0,0,0,0,0,1,0,1,0
15536,43,74,0,1020,0,-3.4,89.93,-27.54,0.74,4882.02,...,0,1,0,0,0,0,0,0,1,0
8819,31,4608,17,480,2,-3.4,95.92,-30.54,2.27,5219.06,...,0,0,1,0,1,0,0,0,0,1
9097,42,376,3,1022,0,1.4,97.58,-40.29,5.01,5322.20,...,0,0,0,0,0,0,0,0,1,0


In [30]:
Xtrain.drop(['job','marital','education','housing','loan','default','poutcome'],inplace=True,axis=1)
Xtrain.head(2)

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed,...,edu_6,edu_7,h_1,h_2,loan_1,loan_2,def_1,def_2,pou_1,pou_2
15632,18,264,1,5,1,-1.7,92.61,-38.37,0.94,5211.23,...,0,1,0,1,0,0,0,0,0,1
13174,27,75,1,959,0,-2.9,91.38,-40.43,1.28,4903.60,...,1,0,0,1,0,0,0,0,1,0


In [28]:
# 哑变量转换 train
dummy_job = pd.get_dummies(Xtrain['job'], prefix='job', drop_first=True)
dummy_mar = pd.get_dummies(Xtrain['marital'], prefix='mar', drop_first=True)
dummy_edu = pd.get_dummies(Xtrain['education'], prefix='edu', drop_first=True)
dummy_h = pd.get_dummies(Xtrain['housing'], prefix='h', drop_first=True)
dummy_loan = pd.get_dummies(Xtrain['loan'], prefix='loan', drop_first=True)
dummy_def = pd.get_dummies(Xtrain['default'], prefix='def', drop_first=True)
dummy_pou = pd.get_dummies(Xtrain['poutcome'], prefix='pou', drop_first=True)
Xtrain = Xtrain.join([dummy_job,dummy_mar,dummy_edu,dummy_h,dummy_loan,
                      dummy_def,dummy_pou])
Xtrain.head(2)

,age,job,marital,education,default,housing,loan,duration,campaign,pdays,...,edu_6,edu_7,h_1,h_2,loan_1,loan_2,def_1,def_2,pou_1,pou_2
15632,18,8,2,7,0,2,0,264,1,5,...,0,1,0,1,0,0,0,0,0,1
13174,27,0,2,6,0,2,0,75,1,959,...,1,0,0,1,0,0,0,0,1,0


In [25]:
from sklearn.model_selection import train_test_split
# 分割
Xtrain, Xtest, Ytrain, Ytest= train_test_split(train2.iloc[:,:-1], train2.iloc[:,-1],
                                               test_size= 0.25,
                                               random_state= 422,
                                               stratify= train2.iloc[:,-1])

In [26]:
Ytrain.value_counts()

0    14661
1     2214
Name: subscribe, dtype: int64

In [20]:
# 数据编码
from sklearn.preprocessing import LabelEncoder
for i in train2.columns:
    if train2[i].dtype == 'object':
        train2[i] = LabelEncoder().fit_transform(train2[i])
train2.head()

,age,job,marital,education,default,housing,loan,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed,subscribe
0,51,0,0,5,0,2,2,4621,1,112,2,0,1.4,90.81,-35.53,0.69,5219.74,0
1,50,7,1,3,1,2,0,4715,1,412,2,1,-1.8,96.33,-40.58,4.05,4974.79,1
2,48,1,0,2,0,0,0,171,0,1027,1,0,-1.8,96.33,-44.74,1.50,5022.61,0
3,26,2,2,3,2,2,2,359,26,998,0,1,1.4,97.08,-35.55,5.11,5222.87,1
4,45,0,2,6,0,0,0,3178,1,240,4,2,-3.4,89.82,-33.83,1.17,4884.70,0


In [19]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22500 entries, 0 to 22499
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               22500 non-null  int64  
 1   job               22500 non-null  object 
 2   marital           22500 non-null  object 
 3   education         22500 non-null  object 
 4   default           22500 non-null  object 
 5   housing           22500 non-null  object 
 6   loan              22500 non-null  object 
 7   duration          22500 non-null  int64  
 8   campaign          22500 non-null  int64  
 9   pdays             22500 non-null  int64  
 10  previous          22500 non-null  int64  
 11  poutcome          22500 non-null  object 
 12  emp_var_rate      22500 non-null  float64
 13  cons_price_index  22500 non-null  float64
 14  cons_conf_index   22500 non-null  float64
 15  lending_rate3m    22500 non-null  float64
 16  nr_employed       22500 non-null  float6

In [11]:
# 删除第一批不用的变量
train2 = train.drop(['id','contact','month','day_of_week',],axis=1)
train2.shape
train2.head()

(22500, 18)

,age,job,marital,education,default,housing,loan,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed,subscribe
0,51,admin.,divorced,professional.course,no,yes,yes,4621,1,112,2,failure,1.4,90.81,-35.53,0.69,5219.74,no
1,50,services,married,high.school,unknown,yes,no,4715,1,412,2,nonexistent,-1.8,96.33,-40.58,4.05,4974.79,yes
2,48,blue-collar,divorced,basic.9y,no,no,no,171,0,1027,1,failure,-1.8,96.33,-44.74,1.50,5022.61,no
3,26,entrepreneur,single,high.school,yes,yes,yes,359,26,998,0,nonexistent,1.4,97.08,-35.55,5.11,5222.87,yes
4,45,admin.,single,university.degree,no,no,no,3178,1,240,4,success,-3.4,89.82,-33.83,1.17,4884.70,no


In [8]:
submission = pd.read_csv('submission.csv')
submission.head(2)

,id,subscribe
0,22501,no
1,22502,no


In [7]:
test = pd.read_csv('test.csv')
test.head()

,id,age,job,marital,education,default,housing,loan,contact,month,...,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed
0,22501,35,technician,single,professional.course,no,yes,yes,cellular,aug,...,3295,1,476,0,success,1.4,95.37,-33.04,3.63,5204.54
1,22502,26,admin.,single,high.school,no,yes,no,cellular,may,...,2872,1,166,2,success,-1.8,91.75,-44.42,3.16,4924.78
2,22503,44,blue-collar,married,basic.6y,no,no,no,telephone,may,...,268,3,968,0,nonexistent,1.1,89.67,-36.90,5.04,4947.02
3,22504,36,blue-collar,married,basic.9y,no,yes,no,telephone,nov,...,30,1,432,5,success,-0.1,89.87,-41.66,3.27,5203.33
4,22505,41,blue-collar,married,basic.4y,no,yes,no,telephone,may,...,1670,1,944,3,success,1.1,97.64,-36.32,3.95,4992.02


In [6]:
train = pd.read_csv('train.csv')
train.head()

,id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_index,cons_conf_index,lending_rate3m,nr_employed,subscribe
0,1,51,admin.,divorced,professional.course,no,yes,yes,cellular,aug,...,1,112,2,failure,1.4,90.81,-35.53,0.69,5219.74,no
1,2,50,services,married,high.school,unknown,yes,no,cellular,may,...,1,412,2,nonexistent,-1.8,96.33,-40.58,4.05,4974.79,yes
2,3,48,blue-collar,divorced,basic.9y,no,no,no,cellular,apr,...,0,1027,1,failure,-1.8,96.33,-44.74,1.50,5022.61,no
3,4,26,entrepreneur,single,high.school,yes,yes,yes,cellular,aug,...,26,998,0,nonexistent,1.4,97.08,-35.55,5.11,5222.87,yes
4,5,45,admin.,single,university.degree,no,no,no,cellular,nov,...,1,240,4,success,-3.4,89.82,-33.83,1.17,4884.70,no


In [5]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')
#设置字体为黑色
plt.rcParams['font.family']='SimHei'
#显示符号
plt.rcParams['axes.unicode_minus']= False

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
import os
os.chdir(r'C:\Users\fzh00\Desktop\文件\excel\阿里天池\银行')